In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam

import matplotlib.pyplot as plt
from torchvision.datasets.mnist import MNIST
from torchvision.transforms import transforms         # 이미지 변형
from torch.utils.data.dataloader import DataLoader  # train - test 분리

from tensorboardX import SummaryWriter
writer = SummaryWriter()

## 1. 데이터 불러오기

In [ ]:
data_transform = transforms.Compose([
    transforms.ToTensor(), # Tensor 데이터 타입으로 변경
    transforms.Resize(32), # 이미지 사이즈를 28 > 32 로 변형 (renet 모델에서 32 사이즈 이미지를 사용하기 때문에)
    transforms.Normalize((0.5),(1.0)) # 평균, 표준편차 | 실전에서는 전체 데이터셋의 평균과 표준편차를 계산하여 넣습니다
])

train_data = MNIST(root='./', train=True, download=True, transform=data_transform) 
test_data = MNIST(root='./', train=False, download=True, transform=data_transform) 
# transform : 데이터 전처리함수

## 2. 데이터 확인하기

In [ ]:
train_data.data.shape

## 3. 배치 사이즈에 따른 데이터 분리

In [5]:
train_loader = DataLoader(train_data, batch_size=32, shuffle=True) 
test_loader = DataLoader(test_data, batch_size=32) 

In [ ]:
data, label = next(iter(train_loader))
print(data.shape)

## 4. 모델 정의하기

In [ ]:
class Lenet(nn.Module): # 📌 모델명(nn.Module) , 괄호 안에 nn.Module 을 반드시 적을 것
    def __init__(self): # 📌 __init__(self) 를 반드시 해야함
        super(Lenet, self).__init__() # 📌 super(모델명, self).__init__() 반드시 해야함

        # convolutions 합성곱을 위한 함수들을 준비한다
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5, stride=1)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5, stride=1)
        self.conv3 = nn.Conv2d(in_channels=16, out_channels=120, kernel_size=5, stride=1)

        # fully connection 
        self.fc1 = nn.Linear(in_features=120, out_features=84)
        self.fc2 = nn.Linear(in_features=84, out_features=10) 
        # ⭐⭐ 마지막 fc함수의 out_features 에는 내가 분류하고 싶은 정답을 적을것!!
    
    # ⭐ 순전파를 하는 함수
    def forward(self, x):  # 📌 forward 안에 self 를 반드시 적어야함
        x = self.conv1(x)                          
        x = F.tanh(x) # 활성함수
        x = F.max_pool2d(x, kernel_size=2, stride=2) # 이미지 사이즈 축소

        x = self.conv2(x)                            
        x = F.tanh(x) # 활성함수
        x = F.max_pool2d(x, kernel_size=2, stride=2) # 이미지 사이즈 축소

        x = self.conv3(x)
        x = F.tanh(x)

        x = torch.reshape(x, (-1, 120)) # 이미지 평탄화

        x = self.fc1(x)             # 선형함수
        x = F.tanh(x)               # 활성함수

        x = self.fc2(x)             # 선형함수
        x = F.tanh(x)               # 활성함수 
        # Renet 모델이 옛날 모델이라 tanh 활성함수를 마지막에 사용 중인데
        # 최근 트렌드는 CrossEntrophy를 손실함수로 사용할 경우 마지막에 활성함수를 사용하지 않는다

        return x

In [18]:
model = Lenet()

## 5. 모델 학습하기

In [ ]:
from torchsummary import summary
summary(model, input_size=(1, 32, 32))

In [ ]:
# 모두 gpu에 올라가게 하는 코드
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)

In [ ]:
from tensorboardX import SummaryWriter
writer = SummaryWriter()

lr = 1e-3                               # ✅ 러닝레이트: 일반적으로 0.001 ~ 0.003
optim = Adam(model.parameters(), lr=lr) # 파라미터 업데이트 최적화 알고리즘
epochs = 10                             # ✅ 학습 반복 횟수
criterion = nn.CrossEntropyLoss()       # ✅ 손실 함수 : 분류모델 - nn.CrossEntropyLoss() / 다른 모델 - nn.MSELoss

step = 0
for epoch in range(epochs):
    # data : 32개의 이미지 데이터 / label : 32개의 이미지의 정답 데이터
    for data, label in train_loader: # [(data, label)]
        optim.zero_grad() # 📌 최적화 함수를 초기화 해야함 (한 번 학습시 마다)

        # ⭐⭐ CNN 의 경우는 이미지의 채널 수만 맞추면 된다
        # Renet 의 경우 1 채널 이미지만 되면 된다!
        # 1) 순전파
        pred = model(data.to(device)) # ⭐데이터 위치 체크

        # 2) 손실 계산
        loss = criterion(pred, label.to(device)) # ⭐데이터 위치 체크

        # 3) 역전파
        loss.backward()
        optim.step() # 4) 파라미터 업데이트

        # tensorboard에 데이터 추가
        writer.add_scalar("Loss/train", loss.item(), step)
        step += 1

    print(f"{epoch + 1} loss : {loss.item()}")

## 6. 모델 저장하기

In [ ]:
# 📌 ⭐⭐⭐ 학습시킨 모델이 날라가지 않도록 반드시 꼭꼭 저장하자
import joblib

# 모델 저장
joblib.dump(model, 'models/number_image_cnn_model.pkl')

## 7. 모델 평가

In [ ]:
# 모델 불러오기
import joblib
model = joblib.load('models/number_image_cnn_model.pkl')

In [ ]:
model.eval() # 📌 모델을 추론용으로 전환하게 하는 코드

# 추가 작성 코드
# falut_data = []

with torch.no_grad():
    total_corr = 0
    for images, labels in test_loader:
        X = images.to(device)
        labels = labels.to(device)

        preds = model(X)
        _, pred = torch.max(preds.data, dim = 1)
        
        result = (pred == labels)
        total_corr += (result).sum().item()

        # 추가 작성 코드
        # for i, re in enumerate(result):
        #     if re == False:
        #         falut_data.append({
        #             'image' : images[i],
        #             'pred' : pred[i],
        #             'label' :labels[i]
        #         })

print(f'정확도 : {total_corr / len(test_data.targets)}')

## 8. 모델 사용

In [ ]:
from torchvision import transforms
from PIL import Image

image_path = 'images/4.jpg'

# 1) 이미지 변환(transform) 파이프라인 정의
# PyTorch 모델에 입력하기 위해 이미지를 텐서로 변환하는 과정을 정의
# ToTensor()는 PIL Image를 PyTorch 텐서로 변환하며, 픽셀 값을 [0, 1] 범위로 정규화
transform = transforms.Compose([
    transforms.ToTensor(),              # 이미지를 PyTorch 텐서로 변환
    transforms.Grayscale(),             # 이미지를 흑백으로 만든다
    transforms.Resize((32, 32)),        # 이미지를 리사이즈
    transforms.Normalize((0.5),(1.0))   # 평균, 표준편차로 정규화 한다
])

with Image.open(image_path) as image:
        print(f"PIL Image 크기: {image.size}")
        print(f"PIL Image 모드: {image.mode}")

        image = image.convert('L')
        
        plt.imshow(image, cmap='gray')
        plt.show()

        # 2) 정의한 변환(transform)을 이미지에 적용
        tensor_image = transform(image)

print("\n이미지를 텐서로 변환 완료:")
print(f"텐서 크기(size): {tensor_image.size}")
print(f"텐서 데이터 타입: {tensor_image.dtype}")

# 3) 모델 예측
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)

# ⭐⭐⭐ 0번째에 강제로 차원을 하나 추가한다
# 왜? 이 모델에는 4차원데이터가 들어가야하니까
# 기존에는 32개의 이미지에 대해서 1채널의 32 x 32 이미지가 담긴
# (32, 1, 32, 32) 형태의 데이터를 모델에 넣었다

# 지금은 1개의 이미지에 대해서 1채널의 32 32 이미지를 모델에 넣고싶으니
# (1, 1, 32, 32) 형태의 데이터를 넣으면 된다
tensor_image = tensor_image.unsqueeze(dim=0)
preds = model(tensor_image.to(device))
_, pred = torch.max(preds.data, dim=1)
print(f"예측 결과: {pred.item()}")